# TensorFlow Hub

[TensorFlow Hub](http://tensorflow.org/hub) is an online repository of already trained TensorFlow models that you can use.
These models can either be used as is, or they can be used for Transfer Learning.

Transfer learning is a process where you take an existing trained model, and extend it to do additional work. This involves leaving the bulk of the model unchanged, while adding and retraining the final layers, in order to get a different set of possible outputs.


# Imports


In [18]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_datasets as tfds


In [19]:
# Load dataset
(raw_train, raw_validation), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    as_supervised=True,  # Returns (image, label) pairs
    with_info=True
)

# Get the number of classes
num_classes = dataset_info.features['label'].num_classes


In [20]:
IMG_SIZE = 224  # Size expected by MobileNet

def preprocess_image(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = preprocess_input(image)  # Normalize image for MobileNet
    return image, label

train = raw_train.map(preprocess_image).shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
validation = raw_validation.map(preprocess_image).batch(32).prefetch(tf.data.AUTOTUNE)


In [21]:
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3))

# Freeze the base model layers
base_model.trainable = False


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')  # Output layer for classification
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenet_1.00_224 (Functional)      │ (None, 7, 7, 1024)          │       3,228,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_2           │ (None, 1024)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 128)                 │         131,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,360,709 (12.82 MB)

 Trainable params: 131,845 (515.02 KB)

 Non-trainable params: 3,228,864 (12.32 MB)

In [25]:
EPOCHS = 10

history = model.fit(
    train,
    validation_data=validation,
    epochs=EPOCHS
)


Epoch 1/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 187s 2s/step - accuracy: 0.5814 - loss: 1.1450 - val_accuracy: 0.8706 - val_loss: 0.3436
Epoch 2/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 172s 2s/step - accuracy: 0.8558 - loss: 0.4113 - val_accuracy: 0.8965 - val_loss: 0.2866
Epoch 3/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 168s 2s/step - accuracy: 0.8946 - loss: 0.2834 - val_accuracy: 0.8856 - val_loss: 0.2677
Epoch 4/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 203s 2s/step - accuracy: 0.9181 - loss: 0.2417 - val_accuracy: 0.9128 - val_loss: 0.2349
Epoch 5/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 213s 2s/step - accuracy: 0.9267 - loss: 0.1879 - val_accuracy: 0.9087 - val_loss: 0.2649
Epoch 6/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 151s 2s/step - accuracy: 0.9361 - loss: 0.1743 - val_accuracy: 0.9142 - val_loss: 0.2353
Epoch 7/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 208s 2s/step - accuracy: 0.9499 - loss: 0.1418 - val_accuracy: 0.9005 - val_loss: 0.2602
Epoch 8/10
92/92 ━━━━━━━━━━━━━━━━━━━━ 206s 2s/step - accuracy: 0.9677 - loss: 0.1160 - val_accuracy: 0.9223 - v

In [26]:
loss, accuracy = model.evaluate(validation)
print(f'Validation Loss: {loss:.4f}, Validation Accuracy: {accuracy:.4f}')


23/23 ━━━━━━━━━━━━━━━━━━━━ 33s 1s/step - accuracy: 0.9183 - loss: 0.2462
Validation Loss: 0.2396, Validation Accuracy: 0.9237


In [30]:
# Save the model to an HDF5 file
model.save('mobilenet_flowers.h5')
